<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/logo.png?raw=true" alt="2022年度ゲノム情報解析入門" height="100px" align="middle">

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>

RNA-seqを用いた解析: ネットワーク解析など
---

　今回は、RNA-seqを用いた解析の中で、遺伝子のネットワーク解析というものを勉強・体験していきます。

また、最近登場した新しいRNA-seqの技術についても少し触れます。

# 今回の勉強内容
　このテキストでは、RNA-seqを用いたネットワーク解析について、および最近のRNA-seq技術について勉強します。

　内容は以下の通りです。

1. 遺伝子ネットワーク解析
  1. 共発現解析
  1. eQTL解析
1. 最近のRNA-seq技術
  1. single cell RNA-seq

---

　一度次のコードセルを実行して、実習に必要なライブラリのインストールや、プログラム、サンプルファイルのダウンロードしてください。

In [ ]:
#########################
##  実習の前に実行してください。 ##
#########################
%%bash
## ソフトウェアのインストール
pip install -q pyvis
## サンプルファイルのダウンロード
wget -q https://github.com/CropEvol/lecture/raw/master/textbook_2022/datasets/RNAseq_demo.tar.gz -O - | tar -zxf - 2>/dev/null

In [ ]:
#########################
##  実習の前に実行してください。 ##
#########################
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pyvis.network import Network
from tqdm import tqdm

# 1. 遺伝子ネットワーク解析

生物はたくさんの遺伝子を持っています。(例えばイネは約30,000の遺伝子をコードしています。)

遺伝子の中には単独で働くものや、他の遺伝子と相互作用することで機能を持つものなど、様々な遺伝子があります。

遺伝子や遺伝子のコードするタンパク質の関係性を表したものをパスウェイと呼びます。

代表的な生物に関しては、(既知のパスウェイに関しては)データベースとして公開されています。

代表的なパスウェイデータベース
* [BioCyc](https://biocyc.org/)
* [KEGG](https://www.genome.jp/kegg/kegg_ja.html)
* [Reactome](https://reactome.org/)
* [WikiPathways](https://www.wikipathways.org/index.php/WikiPathways/)

### 遺伝子ネットワーク

遺伝子の関係性の中でも、遺伝子発現の因果関係(転写制御や促進など)をネットワークとしてモデル化するのが遺伝子のネットワーク解析です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/network.png?raw=true" alt="network" height="280px" align="middle">

遺伝子ネットワークはRNA-seqによる発現量データ等から推定・解析されることが多いです。

共発現解析やeQTL解析と呼ばれる手法は遺伝子ネットワークの推定に利用される手法になります。

今回はこの共発現解析とeQTL解析を学んでいきます。

## 共発現解析

同じパスウェイ上にある機能の類似した遺伝子群や、共通の転写因子によって転写を制御されている遺伝子群などは、遺伝子の発現パターン(発現プロファイルと呼んだりします)が似ていることが多いです。

この現象を共発現と呼びます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression.png?raw=true" alt="coexpression" height="280px" align="middle">

様々な条件下において遺伝子の発現量を測定し、発現プロファイルを確認することで、共発現している遺伝子群を探し、同じパスウェイや転写関係に属する遺伝子群や、関連する機能を持っている遺伝子群を探すことが出来ます。

実際に共発現解析を行う場合、複数の部位やタイムポイント(ある処理をしてから1h後, 2h後, 3h後...etc)でRNA-seqを行い、遺伝子間の発現パターンを比較します。

その上で、似たような発現パターンをしている遺伝子群を共発現しているものとすることが多いです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression2.png?raw=true" alt="coexpression2" height="400px" align="middle">

今回は、前回扱ったイネに病原菌や水を接種した後、複数のタイムポイントでRNA-seqを行った結果を用いて共発現解析をしてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression3.png?raw=true" alt="coexpression3" height="400px" align="middle">

複数のタイムポイントでRNA-seqを行い、発現量をTMM値に変換してあります。

In [ ]:
# 共発現解析のためのデータ
TMM = pd.read_csv("RNAseq_demo/TMM_coex.csv", index_col=0)
TMM

この様なデータを元に、ある遺伝子と遺伝子が共発現しているかを確認します。

何をもって共発現とするのかという所ですが、

代表的なものとしては、着目した２つの遺伝子の発現量の間で相関係数をとり、1に近いほど共発現しているものとする、というやり方が最もシンプルな指標になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression4.png?raw=true" alt="coexpression4" height="250px" align="middle">

In [ ]:
# ある遺伝子と共発現している遺伝子を探す
Gene_A = "Gene.2"
Gene_A_exp = TMM.loc[Gene_A, :]
for i in range(TMM.shape[0]):
    if TMM.index.values[i] != Gene_A:
        Gene_B_exp = TMM.iloc[i, :]
        if np.corrcoef([Gene_A_exp, Gene_B_exp])[0, 1] >= 0.99:
            print(TMM.index.values[i])

In [ ]:
# 発現量をプロットする
Gene_A = "Gene.2"
Gene_B = "Gene.19974"
plt.plot(TMM.loc[Gene_A, :], marker='o', label=Gene_A)
plt.plot(TMM.loc[Gene_B, :], marker='o', label=Gene_B)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0, fontsize=11)
plt.xticks(rotation=60)
plt.show()

ただし、単純な相関係数を基準にすると問題がある場合も多いです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression5.png?raw=true" alt="coexpression5" height="400px" align="middle">

上図では、1つ目の例は共発現していると言ってよさそうですが、

2つ目の例は相関係数としては高い値になっていますが、60h後のデータに引っ張られているだけで、0~48h後のデータは全く相関していないことが分かるかと思います。

単純に相関係数を指標とすると、この様な遺伝子のペアも共発現しているものとしてカウントされるので、注意が必要です。

とはいえ、共発現を決める絶対的な指標は存在しておらず、Mutual Rank(Obayashi et al., 2009)など様々な指標が考案されています。



共発現していると考えられる遺伝子群を特定した後は、その情報をネットワーク図として描写することが多いです。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression6.png?raw=true" alt="coexpression6" height="250px" align="middle">

先ほどのデータから推定できる遺伝子ネットワーク(の一部)を描写してみます。

In [ ]:
# network図の描写
net = Network()

def find_coex_genes(TMM, target):
    target_exp = TMM.loc[target, :]
    coex_genes = []
    for i in range(TMM.shape[0]):
        other_exp = TMM.iloc[i, :]
        if np.corrcoef([target_exp, other_exp])[0, 1] >= 0.99:
            coex_genes.append(TMM.index.values[i])
    return coex_genes

for coex_gene in tqdm(find_coex_genes(TMM, "Gene.2")):
    net.add_node(coex_gene)
    if coex_gene != "Gene.2":
        net.add_edge("Gene.2", coex_gene)
    for coex_coex_gene in find_coex_genes(TMM, coex_gene):
        net.add_node(coex_coex_gene)
        if coex_gene != coex_coex_gene:
            net.add_edge(coex_gene, coex_coex_gene)

net.show("network.html")

Google Colab内のファイルに`network.html`というファイルが生成されているかと思います。このファイルをダウンロードして開くと、ネットワーク図が描写されます。

今回は`Gene.2`と共発現している遺伝子周りのネットワークを図にしただけですが、遺伝子は他にもたくさんあります。

ネットワーク解析を行う際は、一気にすべての遺伝子をネットワークにしようとすると、解析時間が異常にかかってしまったり、図が複雑になりすぎてしまいます。

そのため、着目する遺伝子や遺伝子群に絞って見始める等の工夫が必要です。

## eQTL解析

共発現解析では、似たような発現プロファイルの遺伝子群を発現量のデータから探すことで、同じパスウェイや制御関係にある遺伝子群を探しました。

しかし、共発現解析では、遺伝子群の中でどの遺伝子が上流なのか下流なのかということを決めることは出来ません。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/coexpression7.png?raw=true" alt="coexpression7" height="200px" align="middle">

勿論、既に転写因子としての機能を持つことが知られている様な遺伝子であれば、制御関係を決めることは出来そうですが、大半の遺伝子はまだ機能が分かっていないので現実的ではありません。

そういった場合に、１つの情報を提供する手法となるのがeQTL解析になります。

eQTL解析は少し前にやったQTL解析をRNA-seqに応用したものです。

QTL解析がどんなものだったかというと、交配集団の形質・遺伝子型データ等を用いて、ある形質に影響を与える遺伝子座(量的形質遺伝子座:QTL)を探索する解析でした。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/QTL_example.png?raw=true" alt="QTL_example" height="280px" align="middle">

eQTL解析も同じようにQTLを探す解析ですが、これまでのQTL解析と大きく異なる点は、対象とする形質が葉の長さや籾数等の形質ではなく、遺伝子の発現量を対象にする。という所です。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/eQTL1.png?raw=true" alt="eQTL1" height="320px" align="middle">

つまり、これまでのQTL解析では葉の長さや籾数などに影響を与えている遺伝子座を検出することが目的でしたが、eQTL解析では、ある特定の遺伝子の発現量に影響を与えている遺伝子座を検出することが目的になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/eQTL2.png?raw=true" alt="eQTL2" height="500px" align="middle">

今回はイネのひとめぼれという系統とKALUHEENATIという系統を交配して作成した集団を用いて、eQTL解析をしてみましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/eQTL3.png?raw=true" alt="eQTL3" height="300px" align="middle">

In [ ]:
#eQTL データ
TMM_eQTL = pd.read_csv("RNAseq_demo/TMM_eQTL.csv", index_col=0)
TMM_eQTL

以前GWASを扱ったかと思います。その時は例えば籾数であったり葉の長さを形質として解析を行いました。

eQTL解析では、遺伝子の発現量を形質としてQTL解析を行う形になります。

例えば`Gene.22078`という遺伝子の発現量を形質としてGWASを行ってみましょう。

`Gene.22078`という遺伝子の発現量に影響を与えている遺伝子座(eQTL)を検出することが出来ます。


In [ ]:
# GWAS結果
eQTL_22078 = pd.read_csv("RNAseq_demo/eQTL_22078.csv", index_col=0)
print("特に強く検出された領域")
display(eQTL_22078.loc[eQTL_22078.chr == "chr07", ].sort_values(by="Score",ascending=False).head(3))
display(eQTL_22078.loc[eQTL_22078.chr == "chr10", ].sort_values(by="Score",ascending=False).head(3))
sns.scatterplot(x="Marker", y="Score", hue="chr", data=eQTL_22078, s=10, linewidth=0, legend=False, palette=["blue", "green"]*6)
plt.plot()

この結果から、`Gene.22078`という遺伝子の発現量は、7番染色体の20Mb付近と、10番染色体の7Mb付近のeQTLの遺伝子型によって大きく変化することが分かります。一応Boxplotを描いてみてみると

In [ ]:
# boxplot
genotype = pd.read_csv("RNAseq_demo/eQTL_genotype.csv", index_col=0)
boxplot_data = pd.DataFrame({
    "TMM":TMM_eQTL.loc["Gene.22078", :].values,
    "Genotype1":genotype.loc[:, TMM_eQTL.columns].iloc[0, :],
    "Genotype2":genotype.loc[:, TMM_eQTL.columns].iloc[1, :],
})

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
for col, ax in zip([1, 2], axes):
    p = sns.boxplot(x="Genotype{}".format(col), y="TMM", data=boxplot_data, ax=ax)
    p.set_title("{}_{}".format(genotype.iloc[col-1, 1], genotype.iloc[col-1, 2]))
    p = sns.stripplot(x="Genotype{}".format(col), y='TMM', data=boxplot_data, jitter=True, color='black', ax=ax)
    p.set_xticklabels(['Hitomebore', 'Hetero', 'Kaluheenati'])

この様に、検出されたeQTLの遺伝子型が変わると、発現量が変化することが分かります。

`Gene.22078`という遺伝子は7番染色体の20255488bpに位置しているので、形質とした遺伝子の非常に近い位置と、別の染色体にピークが検出されたことになります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/eQTL_pos.png?raw=true" alt="eQTL_pos" height="280px" align="middle">

eQTLは通常のQTLとは異なり、cis-eQTLとtrans-eQTLという２種類存在していることが知られています。

cis-eQTLは対象としている遺伝子近傍に位置しているeQTLです。

これは、他の遺伝子による影響ではなく、自分自身のプロモーター領域等に変異が入り、発現量が大きく変わっていることを表しています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/cis_eQTL.png?raw=true" alt="cis_eQTL" height="280px" align="middle">

trans-eQTLは対象としている遺伝子と離れた位置にあるeQTLです。

これは、他の遺伝子による影響で発現量が大きく制御されており、その遺伝子に変異が入ることで制御機構が変化したことで起きていると考えられます。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/trans_eQTL.png?raw=true" alt="trans_eQTL" height="280px" align="middle">

つまり、今回得られた結果は`Gene.22078`という遺伝子の発現量を制御する遺伝子が10番染色体の7Mb付近に位置することを示唆しています。この領域に転写因子をコードする遺伝子が位置していれば、可能性は高そうですね。

eQTL解析によって得られる結果は制御関係の上流下流がはっきり決まっているので、共発現解析の時とは異なり、どの遺伝子がどの遺伝子を制御しているのか、という情報に近付きやすいのが利点になります。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/eQTL_benefit.png?raw=true" alt="eQTL_benefit" height="180px" align="middle">

ただし、検出できるのはあくまでも遺伝子座であって遺伝子単体では無いので、検出された領域から候補となる制御遺伝子を見つける方法を別途考える必要はあります。

更に、今回は`Gene.22078`という遺伝子のみの解析結果を見ましたが、イネは約30,000の遺伝子があります。そのため、30,000遺伝子それぞれの発現量に対しeQTL解析を行い、その結果をまとめて考える必要がある、かなり規模の大きな解析になります。



# 最近の新技術について

ここでは最近しばしば活用され始めている新たなRNA-seq技術について触れます。

 ## Single cell RNAseq (scRNA-seq)

 従来のRNA-seqでは、サンプル(組織)をすり潰してRNAを抽出していたので、細胞レベルでは区別されず、まとめてRNAを読んでいた形でした。

そのため、例えば病原菌に感染させた細胞でどの様な遺伝子が発現しているのかを見たい、といった場合にも、採集した組織の中には健全な細胞が混ざっている様な状態でした。

 しかし、近年、Single cell RNA-seqと呼ばれる、細胞レベルでRNA-seqを行うことが可能になってきています。

 参考: [10xGenomicsの説明(scRNAseq)](https://www.10xgenomics.com/blog/single-cell-rna-seq-an-introductory-overview-and-tools-for-getting-started)

 <img src="https://github.com/CropEvol/lecture/blob/master/textbook_2022/images/scRNAseq.png?raw=true" alt="scRNAseq" height="300px" align="middle">



## Spatial Transcriptomics

また、このSingle cell RNA-seqを応用することで、細胞毎に発現量を測定するだけではなく、組織の中でどの部分の細胞がどれだけ発現しているのか、という空間的な情報を調べる様な技術も近年は発展してきています。

参考: [10xGenomicsの説明(Spatial Transcriptomics)](https://www.10xgenomics.com/spatial-transcriptomics)


ただし、まだまだこれらの技術は登場して間もない技術で、シーケンスの値段などが非常に高く、コストがかかる技術です。

 また、これらの技術を用いて何か革命的な発見が出来たのか、と言われると、まだまだそういった結果は出てきていないので、
 
 今後より普及が進み、どの様に扱うと良いか、という部分が洗練されていく必要があるなという印象です。

# まとめ

　今回、遺伝子ネットワーク解析の概要と解析の流れをみてきました。

ネットワーク解析を用いることで、どんな遺伝子が一緒に働いているのか、遺伝子の制御関係がどうなっているのか、といったことを調べることが出来ます。

RNA-seqは実験結果の確認のために発現量を測定する目的で行うこともありますし、RNA-seqのデータから何か新たな発見のヒントを得るために行ったりと、幅広い目的で行われるシーケンス解析です。

皆さんの中でも、RNA-seqデータを使用する人は多いかと思うので、どのような事が出来るかを知っておくと役に立つかもしれません。

<div align="right"><a href="https://github.com/CropEvol/lecture#section2">実習表ページに戻る</a></div>